In [167]:
import os
from glob import glob
import nibabel as nb
import numpy as np
import pandas

In [89]:
jnk='/Users/nalsharif/PING_2017/nifti/t1/p0007_20100128_144417_2_mri.mnc'
wtv=pandas.read_table('/Users/nalsharif/PING_2017/nifti/t1/p0007_20100128_144417_2_mri.txt',delimiter='=')
wtv.index=range(len(wtv))
int(wtv.loc[9][wtv.columns[0]].split()[0].replace('\"',''))
#wtv.ix['\\t\\tstudy:start_date','int study ;']


20100128

In [121]:
nwd='/Users/nalsharif/PING_2017/nifti/t1'
owd='/Users/nalsharif/PING_old/'

In [4]:
sids=[os.path.split(x)[-1][:5] for x in sorted(glob(os.path.join(owd,'P*')))]
sids

['P0007',
 'P0008',
 'P0009',
 'P0010',
 'P0011',
 'P0013',
 'P0014',
 'P0015',
 'P0016',
 'P0017',
 'P0018',
 'P0019',
 'P0022',
 'P0028',
 'P0031',
 'P0032',
 'P0033',
 'P0034',
 'P0035',
 'P0036',
 'P0039',
 'P0040',
 'P0041',
 'P0048',
 'P0053',
 'P0057',
 'P0058',
 'P0061',
 'P0067',
 'P0068',
 'P0071',
 'P0072',
 'P0073',
 'P0074',
 'P0075',
 'P0076',
 'P0077',
 'P0078',
 'P0079',
 'P0085',
 'P0086',
 'P0090',
 'P0091',
 'P0094',
 'P0095',
 'P0096',
 'P0097',
 'P0098',
 'P0099',
 'P0100',
 'P0102',
 'P0103',
 'P0106',
 'P0107',
 'P0108',
 'P0110',
 'P0111',
 'P0118',
 'P0123',
 'P0124',
 'P0125',
 'P0128',
 'P0133',
 'P0136',
 'P0137',
 'P0138',
 'P0139',
 'P0142',
 'P0145',
 'P0148',
 'P0149',
 'P0150',
 'P0151',
 'P0155',
 'P0156',
 'P0157',
 'P0158',
 'P0159',
 'P0160',
 'P0161',
 'P0162',
 'P0163',
 'P0164',
 'P0165',
 'P0166',
 'P0167',
 'P0170',
 'P0171',
 'P0172',
 'P0173',
 'P0174',
 'P0175',
 'P0177',
 'P0178',
 'P0179',
 'P0180',
 'P0181',
 'P0182',
 'P0183',
 'P0189',


```
get list of files in first_nitrc
get list of files in second_piab
1. for img in first_nitrc:
        load image 
        for image2 in second_piab:
                load image
                rotate to same shape
                compare images
                if same
                    exists = true
                    print image:image2 (subjid1 subjid2)
                    break
       if not exist
           print image subjid1 not found in piab
```

In [163]:
nitrc_path = '/Users/nalsharif/PING_old/'
piab_path = '/Users/nalsharif/PING_2017/nifti/t1_nii/'

"""
Multi-line example
"""
# for root, dirs, file in os.walk(nitrc_path):
#     for fl in file:
#         if fl.endswith(".nii.gz") and 'T1' in root:
#             print(os.path.join(root, fl))


"""
Single-line implementation :)
"""
first_nitrc = [os.path.join(root, fl)
               for root, dirs, file in os.walk(nitrc_path) 
               for fl in file
               if fl.endswith(".nii.gz") and 'T1' in root]

second_piab = [os.path.join(root, fl)
               for root, dirs, files in os.walk(piab_path) 
               for fl in files
               if fl.endswith(".nii.gz")]

In [169]:
corr = {}
for nitrc_image in first_nitrc:
    nimg = nb.load(nitrc_image).get_data()
    for piab_image in second_piab:
        pimg = nb.load(piab_image).get_data()

        pimg = pimg.T if nimg.shape != pimg.shape else pimg
        diffy = sum(sum(sum(nimg != pimg)))
        if diffy < 1:
            corr[nitrc_image] = piab_image
            print(nitrc_image, " IS ", corr[nitrc_image])
            break
        else:
#             print("??? {}".format(diffy)),
            continue

print("Summary: {} / {} nitrc files found in piab".format(len(corr.keys()), len(first_nitrc)))
not_found_in_nitrc = second_piab - corr.values()

KeyboardInterrupt: 

In [46]:
df=pandas.DataFrame(index=sids,columns=['use'])
mismatch=[]
missing=[]
dupe=[]
for sid in sids:
    print('working on %s'%sid)
    img=ni.load(glob(os.path.join(owd,'%s*/T1/*/*.nii.gz'%sid))[0]).get_data()
    names=[]
    ovnpaths=sorted(glob(os.path.join(nwd,'p%s*.nii.gz'%sid[1:])))
    if len(ovnpaths) == 0:
        missing.append(sid)
    elif len(ovnpaths) == 1:
        cimg=ni.load(ovnpaths[0]).get_data()
        cimg.reshape((img.shape[0],img.shape[1],img.shape[2]))
        if all(img.flat == cimg.flat):
            df.ix[sid,'use']=ovnpaths[0]
            print('for %s, i found %s'%(sid,ovnpaths[0]))
        else:
            mismatch.append(ovnpaths[0])
    else:
        for pth in ovnpaths:
            cimg=ni.load(pth).get_data()
            cimg.reshape((img.shape[0],img.shape[1],img.shape[2]))
            if all(img.flat == cimg.flat):
                names.append(pth)
        if len(names) == 0:
            mismatch.append(ovnpaths)
        elif len(names) > 1:
            dupe.append(names)
        else:
            df.ix[sid,'use']=names[0]
            print('for %s, i found %s'%(sid,names[0]))
    df.to_csv('/Users/nalsharif/Desktop/df_out.csv')
print('mismatches',mismatch)
print('missing',missing)
print('dupes',dupe)
                

working on P0007
working on P0008
working on P0009
working on P0010


KeyboardInterrupt: 

In [46]:
sids[0]

'P0007'

In [6]:
os.listdir(os.path.join(owd,'%s/T1/*/*.nii.gz'%sid))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/nalsharif/PING_old/P0007/T1/*/*.nii.gz'

In [8]:
sid = 'P0007'
print('working on %s'%sid)
img=ni.load(glob(os.path.join(owd,'%s*/T1/*/*.nii.gz'%sid))[0]).get_data()
names=[]
ovnpaths=sorted(glob(os.path.join(nwd,'p%s*.nii.gz'%sid[1:])))


working on P0007


In [135]:
sid = 'P0007'
print('working on %s'%sid)
#print(glob(os.path.join(owd,'%s*/T1/*/p*/*.txt'%sid)))
hdr1=pandas.read_table(glob(os.path.join(owd,'%s*/T1/*/p*/*.txt'%sid))[0],delimiter='=')
hdr1
#hdr1.index=range(len(hdr1))
#int(hdr1.loc[3][hdr1.columns[0]].split()[0].replace('\"',''))

#glob(os.path.join(nwd,'p%s*.txt'%sid[1:]))
#hdr2=pandas.read_table(glob(os.path.join(nwd,'p%s*.txt'%sid[1:]))[0],delimiter='=')
hdr2
#hdr2.index=range(len(hdr2))
#int(hdr2.loc[3][hdr2.columns[0]].split()[0].replace('\"',''))



working on P0007


,int study ;
\t\tstudy:serial_no,"""0000000619543MR2"" ;"
\t\tstudy:procedure,"""JERNIGAN RESEARCH "" ;"
\t\tstudy:station_id,"""MR01OW2 "" ;"
\t\tstudy:study_id,"""20100128.144417"" ;"
\t\tstudy:software_version,"""14\\LX\\MR Software release:14.0_M5_0737.f"" ;"
\t\tstudy:version,"""MINC Version 1.0"" ;"
\t\tstudy:vartype,"""group________"" ;"
\t\tstudy:institution,"""PU"" ;"
\t\tstudy:modality,"""MRI__"" ;"
\t\tstudy:start_date,"""20100128"" ;"


In [73]:
np.array_equiv(img,e)

False

In [76]:
img.shape==e.shape

True

In [137]:
print(img.shape)
print(cimg.shape)

(256, 256, 166)
(166, 256, 256)


In [144]:
sum(sum(sum(img.T != cimg)))

10826721

In [72]:
a=cimg.transpose(1,2,0)
b=cimg.transpose(2,1,0)
c=cimg.reshape((256,256,166))
d=cimg.reshape((256,256,166),order='F')
e=cimg.reshape((256,256,166),order='A')

In [51]:
cimg.reshape?

In [77]:
all(e.flat==img.flat)

False

In [56]:
print(d,'\n\n')
print(img)

[[[ 21  50 113 ...,  76  70  35]
  [ 22  37 101 ...,  56  77  28]
  [ 12  37 100 ..., 101  73  35]
  ..., 
  [ 25  99 110 ...,  57  45   0]
  [ 37  86 105 ...,  38  34   0]
  [ 47  69  95 ...,  74  33   0]]

 [[ 18  35  99 ...,  67  75  36]
  [ 26  33  61 ...,  60  52  36]
  [  7  31  68 ...,  74  64  39]
  ..., 
  [ 39  93  77 ...,  73  47   0]
  [ 46  80  93 ...,  48  43   0]
  [ 48  61  79 ...,  63  36   0]]

 [[ 14 137  86 ...,  91  50  35]
  [ 19  78  97 ...,  90  71  30]
  [ 20  30  64 ...,  82  87  46]
  ..., 
  [ 31  88  94 ...,  61  40   0]
  [ 58  76  60 ...,  59  43   0]
  [ 37  61  88 ...,  59  34   0]]

 ..., 
 [[ 25  70 105 ...,  68  63  43]
  [ 19  32  79 ...,  80  67  30]
  [ 19 106  86 ...,  96  67  31]
  ..., 
  [ 31  87 111 ...,  56  27   0]
  [ 46  87 121 ...,  64  36   0]
  [ 36  77 140 ...,  56  41   0]]

 [[ 18  55  34 ...,  82  72  33]
  [ 21  31 120 ...,  65  81  40]
  [ 18  82  83 ...,  83  56  39]
  ..., 
  [ 38  56 123 ...,  88  37   0]
  [ 49  58 104 ...,  

In [37]:
print('img:\n',img,'\n')
print('a:\n',a,'\n')
print('b:\n',b)

img:
 [[[24 32 18 ..., 15 15  9]
  [25 34 19 ..., 15 15 10]
  [49 34 42 ..., 28 17 19]
  ..., 
  [33 38 43 ..., 25 18 11]
  [ 0  0  0 ...,  0  0  0]
  [ 0  0  0 ...,  0  0  0]]

 [[20 34 24 ..., 14 12  9]
  [21 37 25 ..., 14 13 10]
  [59 38 50 ..., 24 20 19]
  ..., 
  [35 27 47 ..., 21 19 13]
  [ 0  0  0 ...,  0  0  0]
  [ 0  0  0 ...,  0  0  0]]

 [[26 29 25 ..., 31 22 25]
  [28 31 27 ..., 32 24 26]
  [40 42 40 ..., 58 55 67]
  ..., 
  [37 42 36 ..., 39 48 71]
  [ 0  0  0 ...,  0  0  0]
  [ 0  0  0 ...,  0  0  0]]

 ..., 
 [[19 25 18 ..., 24 12 17]
  [20 30 30 ..., 27 16 21]
  [44 50 47 ..., 54 46 56]
  ..., 
  [45 39 42 ..., 40 41 37]
  [ 6  4  7 ...,  4  4  3]
  [ 5  3  7 ...,  3  4  3]]

 [[14 31 17 ..., 13 20 20]
  [20 32 21 ..., 19 24 25]
  [47 49 53 ..., 46 47 59]
  ..., 
  [47 40 48 ..., 39 42 34]
  [ 4  5  6 ...,  4  4  3]
  [ 3  4  5 ...,  4  3  2]]

 [[21 18 14 ..., 15 16 19]
  [27 16 13 ..., 22 22 23]
  [52 35 32 ..., 67 59 54]
  ..., 
  [57 35 33 ..., 53 37 35]
  [ 6  5  6

In [39]:
a.transpose?